### Notebook to study the best way to architect a db to store wikipedia articles multiple languages

In [1]:
#import py2neo #Libs to connect to neo4j

#def connectDb():
    #"""Function to connect to neo4j database."""

    #py2neo.authenticate("localhost:7474", "neo4j", "lucas")
    #dbConnection = py2neo.Graph("http://localhost:7474/db/data/")

    #return dbConnection

In [2]:
from wikipydia import wikipedia, wikilinks

In [3]:
#art = wikipedia.get_article_by_href("C%2b%2b")
#wikilinks.get_article_links_score(art)

In [4]:
#"Lu'cas".replace("'", "\\'")

In [5]:
from neo4j.v1 import GraphDatabase, basic_auth

from urllib.parse import unquote

class Wiki4Neo():
    def __init__(self, user="neo4j", password="lucas"):
        """Connect to neo4j db"""
        self.driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth(user, password))
        
        #session = driver.session()
        #session.run("CREATE (a:Person {name: {name}, title: {title}})",{"name": "Arthur", "title": "King"})
        #result = session.run("MATCH (a:Person) WHERE a.name = {name} RETURN a.name AS name, a.title AS title",{"name": "Arthur"})
        #for record in result:
        #    print("%s %s" % (record["title"], record["name"]))
        #session.close()

    def save_article(self, article, links, url, lang):
        #Ensure to escape quotes from the title and url before query
        #"Lu'cas".replace("'", "\\'")
        #pageInfo.title = pageInfo.title.replace(/['\\]/g, "\\$&");
        #pageInfo.url = pageInfo.url.replace(/['\\]/g, "\\$&");
        
        article_title = escp(article.title())
        url = escp(url)

        #Construct query
        neo_query = "\n".join([
            'MERGE (article:WikiArticle{{title:"{}", pageId:{}}})'.format(article_title, article.page_id()), 
            'ON CREATE SET article.lang = "{}"'.format(lang), 
            'MERGE (articleUrl:Wikiurl{{url_lang:"{}"}})'.format(url + "_" + lang),
            'ON CREATE SET articleUrl.url = "{}", articleUrl.lang = "{}"'.format(url, lang),
            'SET articleUrl.articleId = {}'.format(article.page_id()),
            "CREATE UNIQUE (articleUrl)-[:RedirectsTo]->(article)"
        ])
        
        #Create queries for links to the article
        for i, (link_href, link_score) in enumerate(links):
            #Ensure to escape quotes from the link before query
            #var link = pageInfo.links[i].replace(/['\\]/g, "\\$&");
            link_href = escp(unquote(link_href)) #Remove url quotes from the href

            neo_query += "\n\n" + "\n".join([
                'MERGE (articleLink{}:Wikiurl{{url_lang: "{}"}})'.format(i, link_href+"_"+lang),
                'ON CREATE SET articleLink{}.url = "{}", articleLink{}.lang = "{}"'.format(i,link_href,i,lang),
                'CREATE UNIQUE (article)-[:LinksTo{{score:{}}}]->(articleLink{})'.format(link_score, i)
            ])
            
        #Create query to create direct connection between articles
        if False:
            neo_query += "\n"

            #Add in/out ConnectsTo relation to this article 
            neo_query += "\n".join([
                "WITH article MATCH (article)-[:LinksTo]->(:Wikiurl)-[:RedirectsTo]->(targetArticle:Article)",
                "CREATE UNIQUE (article)-[:ConnectsTo]->(targetArticle)",
                "WITH article MATCH (targetArticle:Article)-[:LinksTo]->(:Wikiurl)-[:RedirectsTo]->(article)", 
                " CREATE UNIQUE (targetArticle)-[:ConnectsTo]->(article)"
            ])
            
        neo_query += "\nRETURN id(article) as article_id"

        session = self.driver.session()
        results = session.run(neo_query)
        session.close()
        
        for record in results:
            return record["article_id"]

In [6]:
def escp(text):
    return text
    """Function to escape special characters."""
    new_text = text
    new_text = new_text.replace('"', '\\"') #Two bars because one escape the python code and the other the query str
    new_text = new_text.replace("'", "\\'")
    
    return new_text
    
print(escp('test"'))
print(escp("test'"))

test"
test'


In [7]:
w4n = Wiki4Neo()

In [8]:
def save_article_and_links_scores_by_href(href, lang):
    art = wikipedia.get_article_by_href(href, lang)
    links = wikilinks.get_article_links_score(art)
    art_id = w4n.save_article(art, links, href, lang)
    print(art_id)

In [9]:
#save_article_and_links_scores_by_href("c%2b%2b", "en")

In [10]:
#from urllib.parse import quote
#for lang, article in wikipedia.get_article_langlinks("MQTT"):
    #article = quote(article) #quote article title to use as href
    #save_article_and_links_scores_by_href(article, lang)

In [11]:
def get_or_download_pagedata_by_href(href, lang):
    """
    Get data from database, downloading missing data.
    """
    
    #start_time = time.time()
    
    #1. Get the wikiarticle data from the db by the href 
    neo_query = "\n".join([
        'MATCH (n:Wikiurl{{url:"{}", lang:"{}"}})-[:RedirectsTo]->(a:WikiArticle)',
        'RETURN a.title as title, a.pageId as pageid, id(a) as id'
    ]).format(escp(href), lang)
    
    session = w4n.driver.session()
    results = session.run(neo_query)
    session.close()
    
    #print("Time of the 1st try to locate the data:{}".format(time.time() - start_time))
    
    for r in results:
        return r['title'], r['pageid'], r['id']
    
    #start_time = time.time()
    #2. If the href doesn't return an article, download this href
    #Try to get the article
    #If suceed, save the href and try to return the article and create a reference from the href
    art = wikipedia.get_article_by_href(href, lang)
    
    neo_query = "\n".join([
        'MERGE (u:Wikiurl{{url_lang:"{}"}})'.format(escp(unquote(href)) + "_" + lang),
        'ON CREATE SET u.url = "{}", u.lang = "{}", u.articleId = {}'.format(escp(unquote(href)), lang, art.page_id()),
        'WITH u MATCH (a:WikiArticle{{pageId:{}}})'.format(art.page_id()),
        'CREATE UNIQUE (u)-[:RedirectsTo]->(a)',
        'RETURN a.title as title, a.pageId as pageid, id(a) as id'
    ])
    
    session = w4n.driver.session()
    results = session.run(neo_query)
    session.close()
    #print("Time of the 2nd try to locate the data:{}".format(time.time() - start_time))
    
    for r in results:
        return r['title'], r['pageid'], r['id']
    
    #start_time = time.time()
    #3. If nothing has been returned till now, so the article doesn't exists, lets register it
    links = wikilinks.get_article_links_score(art)
    new_art_id = w4n.save_article(art, links, href, lang)
    #print("Time of the 3rd try to locate the data:{}".format(time.time() - start_time))
    
    return art.title(), art.page_id(), new_art_id

In [12]:
#print(get_or_download_pagedata_by_href("c%2b%2b", "en"))

In [13]:
search_langs = [
    'de',
    #'en',
    'fr',
    'es',
    'it',
    'nl',
    'ja',
    'pl',
    'ru',
    'ceb',
    'sv',
    'vi',
    'war'
]

In [14]:
from urllib.parse import quote
import time

from ThreadPool import ThreadPool

def download_and_save_global_articles(href_en):
    start_time = time.time()
    
    #global_articles = list()

    #href_en = "MQTT"
    
    #1. Download english article and get links scores
    #art_en = wikipedia.get_article_by_href(href_en, "en")
    #links_scores = wikilinks.get_article_links_score(art_en)
    #global_articles.append((art_en, links_scores, href_en, "en"))
    #global_article_title = art_en.title()
    #global_article_pageid = art_en.page_id()
    
    #1.1 Get or download english article data
    print("Getting english article data...")
    art_en_title, art_en_page_id, art_en_id = get_or_download_pagedata_by_href(href_en, "en")
    
    global_article_title = art_en_title
    global_article_pageid = art_en_page_id
    
    #2. Get global links by english article title
    print("Getting global links...")
    #global_links = wikipedia.get_article_langlinks(art_en.title())
    #Quote title for correct results for cases of "C++" etc
    global_links = wikipedia.get_article_langlinks(quote(art_en_title)) 
    

    #3. Download articles and generate links scores fot the global links
    #for art_lang, art_title in global_links:
        #art_href = quote(art_title)
        #art = wikipedia.get_article_by_href(art_href, art_lang)
        #art_links_scores = wikilinks.get_article_links_score(art)
        #global_articles.append((art, art_links_scores, art_href, art_lang))
        
    pool = ThreadPool(1)
        
    global_pageids = list()
    global_pageids.append(art_en_id)
    print("Getting global links articles data...") 
    #3.1 Register hrefs from the global links and get their register ids
    
    def _get_page_data(i, total, art_title, art_href, art_lang):
    
        print("{} {} - {}/{}".format(art_title, art_lang, i+1, total))
        try:
            art_title, art_page_id, art_id = get_or_download_pagedata_by_href(art_href, art_lang)
            global_pageids.append(art_id)
        except:
            print("ERROR. Skiping.")
            
        print("Done {} {} - {}/{}".format(art_title, art_lang, i+1, total))
    
    
    for i, (art_lang, art_title) in enumerate(global_links):
        if art_lang not in search_langs:
            continue
            
        art_href = quote(art_title) #get href by quoting title
        
        pool.add_task(_get_page_data, i+1, len(global_links), art_title, art_href, art_lang)
        
        #print("{} {} - {}/{}".format(art_title, art_lang, i+1, len(global_links)))
        #try:
            #art_title, art_page_id, art_id = get_or_download_pagedata_by_href(art_href, art_lang)
            #global_pageids.append(art_id)
        #xcept:
            #print("ERROR. Skiping.")
        
        
    pool.wait_completion()
    
    print(global_pageids)

    #4. Save them into the database and get their ids
    #global_pageids = list()
    #for art in global_articles:
        #new_page_ids = w4n.save_article(art[0], art[1], art[2], art[3])
        #global_pageids.append(new_page_ids)
    
    
    #5. Points every article to the global article of its kind
    print("Saving global articles...")
    neo_query = "\n".join([
        'MERGE (art:GlobalArticle{{title:"{}", en_page_id:{}}})'.format(global_article_title, global_article_pageid),
        "WITH art MATCH (target_art:WikiArticle) WHERE ID(target_art) IN {}".format(global_pageids),
        "CREATE UNIQUE (art)<-[:DerivesFrom]-(target_art)"
    ])    

    session = w4n.driver.session()
    session.run(neo_query)
    session.close()
    print("Done ({} seconds.)".format(time.time() - start_time))

In [15]:
def download_and_set_langlinks(href, lang):
    print(wikipedia.get_article_langlinks(href, lang))

In [16]:
#download_and_set_langlinks("HTML", "en")

In [17]:
#import time

#start_time = time.time()
#get_or_download_pagedata_by_href("MQTT", "pt")
#print(time.time() - start_time)

In [18]:
download_and_save_global_articles("node.js")

Getting english article data...
Getting global links...
Getting global links articles data...
Node.js de - 10/36
Done Node.js de - 10/36
Node.js es - 12/36
Done Node.js es - 12/36
Node.js fr - 16/36
Done Node.js fr - 16/36
Node.js it - 21/36
Done Node.js it - 21/36
Node.js ja - 22/36
Done Node.js ja - 22/36
Node.js nl - 26/36
Done Node.js nl - 26/36
Node.js pl - 28/36
Done Node.js pl - 28/36
Node.js ru - 30/36
Done Node.js ru - 30/36
Node.js sv - 32/36
Done Node.js sv - 32/36
Node.js vi - 36/36
Done Node.js vi - 36/36[3024, 3281, 3339, 3431, 3534, 3553, 3625, 3665, 3703, 3726, 3798]

Saving global articles...
Done (14.523625373840332 seconds.)


In [18]:
query = "\n".join(['MATCH (n:WikiArticle)-[l:LinksTo]->(u:Wikiurl)',
                   'WHERE n.title="Node.js" AND n.lang="en" RETURN u.url AS url, l.score as score'
                  ])

session = w4n.driver.session()
results = session.run(query)
session.close()


urls_to_go = list()
higher_score = 0
for r in results:
    higher_score = max(higher_score, r['score'])
    urls_to_go.append((r['url'], r['score']))

In [19]:
urls_to_go

[('Integrated_development_environment', 3.0),
 ('C_(programming_language)', 3.0),
 ('PHP', 3.0),
 ('V8_JavaScript_engine', 3.0),
 ('HTTP', 3.0),
 ('Unix', 3.0),
 ('Joyent', 3.0),
 ('Package_manager', 3.0),
 ('Open-source_software', 3.0),
 ('Web_application', 3.0),
 ('Web_server', 4.0),
 ('Linux', 4.0),
 ('Microsoft_Windows', 5.0),
 ('Application_programming_interface', 4.0),
 ('Libuv', 5.999999999999998),
 ('Google', 5.0),
 ('Microsoft', 6.0),
 ('Npm_(software)', 6.999999999999997),
 ('Input/output', 7.0),
 ('Server-side', 7.0),
 ('Event_loop', 8.0),
 ('Asynchronous_I/O', 8.0),
 ('Callback_(computer_programming)', 10.0),
 ('V8_(JavaScript_engine)', 10.0),
 ('JavaScript', 25.0),
 ('Context_switch', 1.0),
 ('Non-blocking_I/O', 1.0),
 ('Vertical_scaling', 1.0),
 ('Observer_pattern', 1.0),
 ('Koding', 1.0),
 ('Cloud9_IDE', 1.0),
 ('GitHub', 1.0),
 ('Node-RED', 1.0),
 ('Lodash', 1.0),
 ('CommonJS', 1.0),
 ('PostgreSQL', 1.0),
 ('Grunt_(software)', 1.0),
 ('Google_Chrome', 1.0),
 ('CPU_core'

In [ ]:
for i, (url, score) in enumerate(urls_to_go):
    #Skip less relevant stuff
    if score / higher_score < 0.1:
        continue

    print("WORKING ON {} {}/{}".format(url, i+1, len(urls_to_go)))
    try:
        download_and_save_global_articles(quote(url))
    except:
        print("ERROR. SKIPING URL")

WORKING ON Integrated_development_environment 1/139
Getting english article data...
Getting global links...
Getting global links articles data...Integrierte Entwicklungsumgebung de - 11/45Entorno de desarrollo integrado es - 14/45Environnement de développement fr - 17/45Integrated development environment it - 22/45統合開発環境 ja - 23/45Integrated development environment nl - 29/45Zintegrowane środowisko programistyczne pl - 31/45Интегрированная среда разработки ru - 34/45Integrerad utvecklingsmiljö sv - 40/45Môi trường phát triển tích hợp vi - 45/45










Done Entorno de desarrollo integrado es - 14/45
Done Integrated development environment it - 22/45
Done Integrated development environment nl - 29/45
Done Môi trường phát triển tích hợp vi - 45/45
Done Integrerad utvecklingsmiljö sv - 40/45
Done Zintegrowane środowisko programistyczne pl - 31/45
Done 統合開発環境 ja - 23/45
Done Environnement de développement fr - 17/45
Done Интегрированная среда разработки ru - 34/45
Done Integrierte Entwic

Done (11.889679908752441 seconds.)
WORKING ON Linux 12/139
Getting english article data...
Getting global links...
Getting global links articles data...
Linux ceb - 24/138Linux it - 55/138Linux ja - 56/138Linux nl - 86/138Linux fr - 40/138Linux vi - 126/138Linux de - 31/138Linux sv - 112/138Linux pl - 92/138Linux ru - 97/138









Done Linux ceb - 24/138
Linux war - 129/138
Done Linux pl - 92/138
Done Linux ja - 56/138
Done Linux it - 55/138Done Linux vi - 126/138

Done Linux fr - 40/138
Done Linux sv - 112/138
Done Linux de - 31/138Done Linux war - 129/138

Done Linux nl - 86/138Done Linux ru - 97/138

[12124, 12560, 13413, 13205, 13076, 13910, 12735, 13642, 12561, 13973, 13379, 13499]
Saving global articles...
Done (13.90279507637024 seconds.)
WORKING ON Microsoft_Windows 13/139
Getting english article data...
Getting global links...
Getting global links articles data...
Microsoft Windows ceb - 20/130Microsoft Windows de - 26/130Microsoft Windows es - 30/130Microsoft Windows fr - 

In [19]:
query = "\n".join(['MATCH (n:GlobalArticle)<-[:DerivesFrom]-(:WikiArticle)-[l:LinksTo]->(u:Wikiurl)',
                   'WHERE n.title = "Node.js" RETURN l.score as score,u.url as url, u.lang as lang'
                  ])

session = w4n.driver.session()
result = session.run(query)
session.close()


#Get results and fil the lang links list
from collections import defaultdict, Counter

lang_links = defaultdict(list)

for r in result:
    lang_links[r['lang']].append([r['url'], r['score']])
    
    
    
#Normalize scores for every lang
for lang in lang_links.keys():
    score_sum = 0
    for url, score in lang_links[lang]:
        score_sum += score
    
    for i,(url, score) in enumerate(lang_links[lang]):
        lang_links[lang][i].append(score/score_sum)

In [21]:
#Accumulate everything (assuming urls share href among languages)
#Maybe weight things by the number urls they have
#And get only things that the english version links to
result_score = Counter()

for lang in lang_links.keys():
    for url, score, score_norm in lang_links[lang]:
        print(url)
        result_score[url] += score_norm

result_score.most_common()

マイクロソフト
インターネット・リレー・チャット
Freenode
ディレクトリ
ミドルウェア
ウェブアプリケーション
機能拡張
ストリーム_(プログラミング)
タイマー
バッファ
ファイルシステム
Transmission_Control_Protocol
エコー_(コンピュータ)
ポート番号
Ruby
ウェブブラウザ
Perl
C言語
Python
Twisted
プログラム_(コンピュータ)
2009年
サーバーサイド・スクリプト
スケーラビリティ
イベント駆動型プログラミング
入出力
ソフトウェアフレームワーク
Express.js
コンパイラ
ファイル_(コンピュータ)
Joyent
コンピュータネットワーク
バイナリ
Hello_world
プラットフォーム_(コンピューティング)
Unix系
Webサーバ
Google_V8_JavaScript_Engine
Microsoft_Windows
CommonJS
サードパーティー
MEAN_(ソフトウェアバンドル)
JavaScript
モジュール
서버
Echo
트위스티드_(소프트웨어)
루비_(프로그래밍_언어)
플리커
Npm
마이크로소프트
마이크로소프트_윈도
포트_번호
전송_제어_프로토콜
웹_서버
자바스크립트
V8_(자바스크립트_엔진)
입출력
소프트웨어_프레임워크
펄
Hello_world
파이썬
Protokol_Kawalan_Penghantaran
JavaScript
Pelayan_web
Sistem_perisian
Pustaka_(pengkomputeran)
Google
JavaScript
HyperText_Transfer_Protocol
Microsoft_Windows
Hello_world_(programma)
Linux
Mac_OS_X
Python_(programmeertaal)
Web_server
J2EE
ASP.NET
Apache_(webserver)
Server-side_scripting
PHP
Perl_(programmeertaal)
Ruby_On_Rails
Common_Gateway_Interface
Linux_on_System_z
Asynkron_kommunikasjon
P

[('JavaScript', 5.095364250903193),
 ('जावास्क्रिप्ट', 1.0),
 ('אתר_רשמי', 0.9631236442516269),
 ('Wikimedia_Commons', 0.9186585542517746),
 ('C_(ngôn_ngữ_lập_trình)', 0.627906976744186),
 ('Microsoft', 0.5844800742560157),
 ('Web_application', 0.5107913669064749),
 ('سەرچاوە_کراوە', 0.5),
 ('ئینگلیزی', 0.5),
 ('Google', 0.4705846742475262),
 ('Microsoft_Windows', 0.41089584337771046),
 ('Linux', 0.3917358083969188),
 ('HTTP', 0.37156830245951505),
 ('Wayback_Machine', 0.3333333333333333),
 ('Internet', 0.3333333333333333),
 ('Program', 0.3333333333333333),
 ('زبان_انگلیسی', 0.2800000000000002),
 ("Камп'ютарная_праграма", 0.26666666666666666),
 ('Yahoo!', 0.2559944060956666),
 ('HyperText_Transfer_Protocol', 0.25),
 ('Http', 0.24242424242424243),
 ('C++', 0.2366988957509398),
 ('서버', 0.23076923076923078),
 ('PHP', 0.20889150783753271),
 ('سی_(پروگرامنگ_زبان)', 0.2),
 ('Runtime', 0.18181818181818182),
 ('OS_X', 0.17822541116376422),
 ('Server', 0.17565485362095531),
 ('Transmission_Cont

In [171]:
eng_counter = Counter()
for url, score, score_norm in lang_links["en"]:
    eng_counter[url] += score_norm
    
eng_counter.most_common()

[('C_(programming_language)', 0.2324324324324324),
 ('Standardization', 0.05945945945945945),
 ('International_Organization_for_Standardization', 0.03378378378378378),
 ('Library_(computing)', 0.03378378378378378),
 ('Feature_(software_design)', 0.03243243243243243),
 ('Generic_programming', 0.03175675675675675),
 ('Operator_(programming)', 0.024324324324324322),
 ('Exception_handling', 0.02027027027027027),
 ('Class_(computer_programming)', 0.018018018018018025),
 ('C++14', 0.014864864864864867),
 ('Compile-time', 0.014864864864864862),
 ('Template_(programming)', 0.012837837837837835),
 ('C++11', 0.012162162162162168),
 ('Base_class', 0.012162162162162161),
 ('Variable_(programming)', 0.01081081081081081),
 ('Inheritance_(computer_science)', 0.01081081081081081),
 ('Inheritance_(object-oriented_programming)', 0.01081081081081081),
 ('Data', 0.01081081081081081),
 ('Performance_(software)', 0.009459459459459458),
 ('Compiler', 0.009459459459459458),
 ('Operator_overloading', 0.0094594

In [28]:
#wikipedia.get_article_langlinks("Artificial_neural_network")

In [134]:
#save_article_and_links_scores_by_href("c%2b%2b", "he")

In [131]:
#"ב".encode(encoding='ascii',errors='strict').decode("utf-8", errors="strict") 

In [132]:
#wikipedia.get_article_langlinks("C%2b%2b")

In [133]:
#MATCH (n:GlobalArticle)<-[:DerivesFrom]-(:WikiArticle)-[l:LinksTo]->(u:Wikiurl)-[:RedirectsTo]->(:WikiArticle)-[:DerivesFrom]->(o:GlobalArticle) WHERE n.title = "MQTT" RETURN n,u,l,o